In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor, plot_tree
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
import category_encoders as ce

In [20]:
df = pd.read_csv(r'/Users/lauverm/dat-11-15/ClassMaterial/Unit1/data/master.csv', parse_dates=['visit_date'])

df['month'] = df['visit_date'].dt.month
df['year']  = df['visit_date'].dt.year
df['quarter'] = df['visit_date'].dt.quarter

df = df.fillna(0)



In [21]:
from sklearn.pipeline import make_pipeline

In [41]:
tree = DecisionTreeRegressor(max_depth = 6)

In [23]:
te = ce.TargetEncoder()

In [24]:
df.sort_values(by=['id','visit_date'],inplace=True)

In [25]:
train = df.groupby('id').apply(lambda x: x.iloc[:-15])
test  = df.groupby('id').apply(lambda x: x.iloc[-15:])

In [44]:
X_train = train.drop(['visit_date', 'visitors'], axis=1)
y_train = train['visitors']

X_test  = test.drop(['visit_date','visitors'], axis=1)
y_test = test['visitors']

pipe = make_pipeline(te, tree)

pipe.fit(X_train, y_train).score(X_test, y_test)

0.4623630680882702

In [45]:
pipe.score(X_train, y_train)

0.46232321445164726

In [47]:
train = train.reset_index(drop = True)

val   = train.groupby('id').apply(lambda x: x.iloc[-15:])
train = train.groupby('id').apply(lambda x: x.iloc[:-15])

In [48]:
X_train = train.drop(['visit_date', 'visitors'], axis=1)
y_train = train['visitors']

X_val  = val.drop(['visit_date','visitors'], axis=1)
y_val = val['visitors']

In [59]:
pipe[-1].set_params(max_depth = 11)
pipe.fit(X_train, y_train).score(X_val, y_val)

0.5249206658458627

In [61]:
def get_val_scores(df, estimator):
    
    df = df.drop('visit_date', axis = 1)
    
    # create training and validation set
    train = df.groupby('id').apply(lambda x: x.iloc[:-15]).reset_index(drop = True)
    val   = df.groupby('id').apply(lambda x: x.iloc[-15:]).reset_index(drop = True)
    
    # create a validation & test set
    X_train, y_train = train.drop('visitors', axis = 1), train['visitors']
    X_val, y_val     = val.drop('visitors', axis = 1), val['visitors']
    
    estimator.fit(X_train, y_train)
    
    # score on the test data
    return estimator.score(X_val, y_val)



In [60]:
train = df.groupby('id').apply(lambda x: x.iloc[:-15]).reset_index(drop=True)
test  = df.groupby('id').apply(lambda x: x.iloc[-15:]).reset_index(drop=True)

In [62]:
get_val_scores(train,pipe)

0.5245086185017496

In [63]:
pipe[-1].set_params(max_depth = 6)

DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [65]:
train['month']           = train['visit_date'].dt.month
train['year']            = train['visit_date'].dt.year
train['passage_of_time'] = (df['visit_date'] - df['visit_date'].min()).dt.days

In [66]:
train

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,year,quarter,passage_of_time
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,12
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,7,2016,3,13
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,14
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,15
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239668,air_fff68b929994bfbd,2017-04-03,2,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2,215
239669,air_fff68b929994bfbd,2017-04-04,4,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2,216
239670,air_fff68b929994bfbd,2017-04-05,6,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0,4,2017,2,217
239671,air_fff68b929994bfbd,2017-04-06,6,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0,4,2017,2,218


In [67]:
get_val_scores(train,pipe)

0.4794210840577293

In [70]:
train['yesterday'] = train.groupby('id')['visitors'].shift()
#called autoregresion

In [73]:
train.bfill()

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,year,quarter,passage_of_time,yesterday
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,12,35.0
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,7,2016,3,13,35.0
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,14,9.0
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,15,20.0
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,17,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239668,air_fff68b929994bfbd,2017-04-03,2,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2,215,2.0
239669,air_fff68b929994bfbd,2017-04-04,4,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2,216,2.0
239670,air_fff68b929994bfbd,2017-04-05,6,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0,4,2017,2,217,4.0
239671,air_fff68b929994bfbd,2017-04-06,6,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0,4,2017,2,218,6.0


In [74]:
train['two_days_ago'] = train.groupby('id')['visitors'].shift(2)
train['three_days_ago'] = train.groupby('id')['visitors'].shift(3)
train['one_week_ago'] = train.groupby('id')['visitors'].shift(7)
train['one_month_ago'] = train.groupby('id')['visitors'].shift(30)

In [82]:
del train['three_week_ago']

In [76]:
train.bfill(inplace=True)

In [77]:
train

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,year,quarter,passage_of_time,yesterday,two_days_ago,three_week_ago,one_week_ago,one_month_ago
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,12,35.0,35.0,35.0,35.0,35.0
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,7,2016,3,13,35.0,35.0,35.0,35.0,35.0
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,14,9.0,35.0,35.0,35.0,35.0
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,15,20.0,9.0,35.0,35.0,35.0
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3,17,25.0,20.0,9.0,35.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239668,air_fff68b929994bfbd,2017-04-03,2,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2,215,2.0,9.0,5.0,4.0,5.0
239669,air_fff68b929994bfbd,2017-04-04,4,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2,216,2.0,2.0,9.0,9.0,7.0
239670,air_fff68b929994bfbd,2017-04-05,6,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0,4,2017,2,217,4.0,2.0,2.0,2.0,7.0
239671,air_fff68b929994bfbd,2017-04-06,6,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0,4,2017,2,218,6.0,4.0,2.0,5.0,5.0


In [78]:
get_val_scores(train,pipe)

0.4912491788355623

In [87]:
train['three_day_average']= train.groupby('id')['visitors'].rolling(3).mean().shift().values

In [88]:
train['seven_day_average'] = train.groupby('id')['visitors'].rolling(7).mean().shift().values
train['thirty_day_average'] = train.groupby('id')['visitors'].rolling(30).mean().shift().values
#need shift here because of look ahead bias. Need values because of index.  

In [92]:
train['ninety_day_average'] = train.groupby('id')['visitors'].rolling(90).mean().shift().values

In [93]:
train.bfill(inplace=True)

In [94]:
get_val_scores(train,pipe)

0.5084694600336774

In [95]:
pipe

Pipeline(memory=None,
         steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'genre', 'area'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', min_samples_leaf=1,
                               return_df=True, smoothing=1.0, verbose=0)),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(criterion='mse', max_depth=6,
                                       max_features=None, max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort=False, random_state=None,
                                       splitter='best'))],
         verbose=False)

In [96]:
pipe[-1].feature_importances_

array([3.52749078e-02, 8.77681016e-02, 2.00318733e-03, 0.00000000e+00,
       0.00000000e+00, 6.36624798e-04, 0.00000000e+00, 0.00000000e+00,
       1.96821780e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.16852743e-03, 5.87665017e-04, 2.55960981e-02, 0.00000000e+00,
       0.00000000e+00, 1.61930442e-02, 8.25409582e-01, 1.39404401e-03])

In [100]:
pipe[0].transform(train)

ValueError: Unexpected input dimension 22, expected 20

In [102]:
X_train, y_train = train.drop(['visitors', 'visit_date'], axis = 1), train['visitors']


In [105]:
train_cols = pipe[2].transform(pipe[1].transform(pipe[0].transform(X_train))).columns

IndexError: list index out of range

In [ ]:
pd.DataFrame({
    'Col': train_cols,
    'Imp': 
})

In [106]:
df

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,year,quarter
166836,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3
166837,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,7,2016,3
166838,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3
166839,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3
166840,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,7,2016,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216643,air_fff68b929994bfbd,2017-04-18,6,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2
216644,air_fff68b929994bfbd,2017-04-19,2,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,2
216645,air_fff68b929994bfbd,2017-04-20,2,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,1.0,4,2017,2
216646,air_fff68b929994bfbd,2017-04-21,4,Friday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,6.0,4,2017,2


In [108]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239673 entries, 0 to 239672
Data columns (total 22 columns):
id                    239673 non-null object
visit_date            239673 non-null datetime64[ns]
visitors              239673 non-null int64
day_of_week           239673 non-null object
holiday               239673 non-null int64
genre                 239673 non-null object
area                  239673 non-null object
latitude              239673 non-null float64
longitude             239673 non-null float64
reserve_visitors      239673 non-null float64
month                 239673 non-null int64
year                  239673 non-null int64
quarter               239673 non-null int64
passage_of_time       239673 non-null int64
yesterday             239673 non-null float64
two_days_ago          239673 non-null float64
one_week_ago          239673 non-null float64
one_month_ago         239673 non-null float64
three_day_average     239673 non-null float64
seven_day_average     23

In [110]:
train

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,...,quarter,passage_of_time,yesterday,two_days_ago,one_week_ago,one_month_ago,three_day_average,seven_day_average,thirty_day_average,ninety_day_average
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,...,3,12,35.0,35.0,35.0,35.0,21.333333,27.714286,26.466667,24.622222
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,...,3,13,35.0,35.0,35.0,35.0,21.333333,27.714286,26.466667,24.622222
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,...,3,14,9.0,35.0,35.0,35.0,21.333333,27.714286,26.466667,24.622222
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,...,3,15,20.0,9.0,35.0,35.0,21.333333,27.714286,26.466667,24.622222
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,...,3,17,25.0,20.0,35.0,35.0,18.000000,27.714286,26.466667,24.622222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239668,air_fff68b929994bfbd,2017-04-03,2,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,...,2,215,2.0,9.0,4.0,5.0,5.333333,5.142857,5.066667,4.944444
239669,air_fff68b929994bfbd,2017-04-04,4,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,...,2,216,2.0,2.0,9.0,7.0,4.333333,4.857143,4.966667,4.933333
239670,air_fff68b929994bfbd,2017-04-05,6,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0,...,2,217,4.0,2.0,2.0,7.0,2.666667,4.142857,4.866667,4.933333
239671,air_fff68b929994bfbd,2017-04-06,6,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0,...,2,218,6.0,4.0,5.0,5.0,4.000000,4.714286,4.833333,4.944444


In [109]:
train.drop(['two_days_ago','three_days_ago','one_month_ago','three_day_average','seven_day_average','thirty_day_average','ninety_day_average'],inplace=True)

KeyError: "['two_days_ago' 'three_days_ago' 'one_month_ago' 'three_day_average'\n 'seven_day_average' 'thirty_day_average' 'ninety_day_average'] not found in axis"

In [111]:
max_depth = [5,6,7]
max_features = [0.6,0.8,1]
min_samples_leaf = [5,20,40]

cv_scores = []

for depth in max_depth:
    for feature in max_features:
        for sample in min_samples_leaf:
            print(f"Getting validation score for: depth: {depth}, max_features: {feature}, leaf_samples: {sample}")
            pipe[-1].set_params(max_depth = depth, max_features = feature, min_samples_leaf = sample)
            val_score = get_val_scores(train, pipe)
            cv_scores.append((val_score, depth, feature, sample))
            

Getting validation score for: depth: 5, max_features: 0.6, leaf_samples: 5
Getting validation score for: depth: 5, max_features: 0.6, leaf_samples: 20
Getting validation score for: depth: 5, max_features: 0.6, leaf_samples: 40
Getting validation score for: depth: 5, max_features: 0.8, leaf_samples: 5
Getting validation score for: depth: 5, max_features: 0.8, leaf_samples: 20
Getting validation score for: depth: 5, max_features: 0.8, leaf_samples: 40
Getting validation score for: depth: 5, max_features: 1, leaf_samples: 5
Getting validation score for: depth: 5, max_features: 1, leaf_samples: 20
Getting validation score for: depth: 5, max_features: 1, leaf_samples: 40
Getting validation score for: depth: 6, max_features: 0.6, leaf_samples: 5
Getting validation score for: depth: 6, max_features: 0.6, leaf_samples: 20
Getting validation score for: depth: 6, max_features: 0.6, leaf_samples: 40
Getting validation score for: depth: 6, max_features: 0.8, leaf_samples: 5
Getting validation scor

In [113]:
max(cv_scores)

(0.5151934264247477, 7, 0.6, 40)

In [114]:
pipe[-1].set_params(max_depth = 7, max_features = 0.8, min_samples_leaf = 40)

DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=0.8,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=40,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [116]:
df['yesterday'] = df.groupby('id')['visitors'].shift()
df['one_week_ago'] = df.groupby('id')['visitors'].shift(7)

df=df.bfill()

In [117]:
get_val_scores(df, pipe)

0.4910588900442545